In [1]:
import java.nio.file.Files
import java.nio.file.Path

data class BenchmarkData(val name: String, val score: Double, val error: Double) {
    val type = getType(name)
    override fun toString(): String = "$name: $score ± $error ns/op"
}

enum class Type { Int, Long, Float, Double }

fun getType(name: String) = when {
    name.contains("Float") -> Type.Float
    name.contains("Int") -> Type.Int
    name.contains("Double") -> Type.Double
    name.contains("Long") -> Type.Long // Long must be the last as it is contained in other test names
    else -> error("Unknown type: ${name}")
}

fun readHotspotBenchmarkData(benchmarkName: String) = Files.readAllLines(Path.of("build", "results", "jmh", "results.txt"))
    .mapNotNull {
        val r = Regex("$benchmarkName\\.([^ ]+) +avgt +\\d+ +([0-9.]+) +± +([0-9.]+) +ns/op")
        val match = r.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            name = match.groups[1]!!.value,
            score = match.groups[2]!!.value.toDouble(),
            error = match.groups[3]!!.value.toDouble(),
        )
    }.sortedBy { it.type }
(readHotspotBenchmarkData("Wolf3dBenchmark") + readHotspotBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineFloat: 3.2777794122E7 ± 117599.149 ns/op
inlineFloat: 3.2454402942E7 ± 71914.263 ns/op
longPackFloat: 5.6637237703E7 ± 30654.969 ns/op
mutableSpecificRefFloat: 2.3727817417E7 ± 38906.477 ns/op
mutableUniversalRefFloatIn1Long: 6.3879335429E7 ± 207183.718 ns/op
mutableUniversalRefFloatIn2Longs: 3.8662847293E7 ± 62597.21 ns/op
valueFloat: 2.4604456636E7 ± 363608.407 ns/op
valueInlineFloat: 2.4917343507E7 ± 35561.195 ns/op
valuePreserveBoxFloat: 2.5026109333E7 ± 75842.585 ns/op
baselineDouble: 3.3724536015E7 ± 90546.797 ns/op
inlineDouble: 3.4364489472E7 ± 204342.15 ns/op
mutableSpecificRefDouble: 2.3247218375E7 ± 37047.305 ns/op
mutableUniversalRefDouble: 3.942429982E7 ± 65405.138 ns/op
valueDouble: 2.4616159057E7 ± 108055.95 ns/op
valueInlineDouble: 2.4570258084E7 ± 157137.427 ns/op
valuePreserveBoxDouble: 2.4515243674E7 ± 99634.239 ns/op

In [2]:
%use lets-plot

In [3]:
import org.jetbrains.letsPlot.intern.PosKind
import org.jetbrains.letsPlot.intern.StatKind
import org.jetbrains.letsPlot.intern.layer.PosOptions
import org.jetbrains.letsPlot.intern.layer.StatOptions

fun makePlot(name: String, platform: String, benchmarkLines: List<BenchmarkData>) {
    if (benchmarkLines.isEmpty()) return
    val names = benchmarkLines.map { it.name }
    val scores = benchmarkLines.map { it.score }
    val scoresMinusErrors = benchmarkLines.map { it.score - it.error / 2 }
    val scoresPlusErrors = benchmarkLines.map { it.score + it.error / 2 }
    val baseline = benchmarkLines.filter { it.name.startsWith("baseline") }.groupBy { it.type }.mapValues { (_, v) -> v.single() }
    val maxScore = scoresPlusErrors.max()
    val plot = letsPlot() +
            labs("$name ($platform)", fill = "Benchmark", y = "ns/op") +
            geomBar(stat = StatOptions(kind = StatKind.IDENTITY)) {
                x = names
                y = scores
                fill = names
            } +
            geomErrorBar(
                width = 0.5,
                position = PosOptions(kind = PosKind.DODGE),
                size = 0.6
            ) {
                x = names
                ymin = scoresMinusErrors
                ymax = scoresPlusErrors
            } +
            geomText(
                stat = StatOptions(kind = StatKind.IDENTITY),
                position = positionStack(vjust = 1.0),
                size = 8.0 - names.size.toDouble() / 5,
            ) {
                x = names
                y = scores.map {
                    val threshold = 0.09
                    if (it < maxScore * threshold) it + threshold * maxScore / 2 else it / 2
                }
                label = benchmarkLines.map {
                    val score = it.score / baseline[it.type]!!.score
                    val error = it.error / baseline[it.type]!!.score
                    "${String.format("%.0f", score * 100)}%\n±\n${String.format("%.1f", error * 100)}%\n${it.type}"
                }
            } +
            scaleXDiscrete(
                labels = names,
                name = "",
                limits = names + listOf(""),
                breaks = names.toList()
            ) +
            scaleYContinuous(limits = 0 to maxScore, name = "ns/op") +
            theme(plotTitle = elementText(hjust = 0.5)).legendPositionRight() +
            ggsize(1000, 800)
    ggsave(plot = plot, filename = "$name ($platform).svg")
    plot.show()
}


In [4]:
makePlot("Wolf3d", "Hotspot", readHotspotBenchmarkData("Wolf3dBenchmark"))
makePlot("Ackermann", "Hotspot", readHotspotBenchmarkData("AckermannBenchmark"))
makePlot("BoxRecreation", "Hotspot", readHotspotBenchmarkData("BoxRecreationBenchmark"))

In [5]:
import java.io.FileFilter
import kotlin.io.path.listDirectoryEntries

fun readArtBenchmarkData(benchmarkName: String) = Path.of("android-benchmark/build/outputs/androidTest-results/connected")
    .toFile().listFiles(FileFilter { it.isDirectory })!!.single()
    .listFiles { dir, name -> name.startsWith("logcat-org.jetbrains.") }!!.flatMap { it.readLines() }
    .mapNotNull {
        val pattern = Regex("^\\d{2}-\\d{2} +\\d+:\\d+:\\d+.\\d+ +\\d+ +\\d+ +I +Benchmark: +$benchmarkName\\.([^\\[]+)\\[Metric \\(timeNs\\) +results: +median +([^,]+), +min [^,]+, +max [^,]+, +standardDeviation: +([^,]+), +.*")
        val match = pattern.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            name = match.groups[1]!!.value,
            score = match.groups[2]!!.value.toDouble(),
            error = match.groups[3]!!.value.toDouble(),
        )
     }.let { lines ->
         val indexes = readHotspotBenchmarkData(benchmarkName).mapIndexed { index, benchmarkData -> benchmarkData.name to index }.toMap()
         if (lines.all { it.name in indexes }) lines.sortedBy { indexes[it.name]!! } else lines.sortedBy { it.type }
     }
(readArtBenchmarkData("Wolf3dBenchmark") + readArtBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineFloat: 3.134010385E8 ± 8248733.538496841 ns/op
inlineFloat: 6.662220955E8 ± 1.1067735643873254E7 ns/op
longPackFloat: 7.77690005E7 ± 945778.9748266232 ns/op
mutableSpecificRefFloat: 4.75664325E7 ± 510325.65365526336 ns/op
mutableUniversalRefFloatIn1Long: 1.596548075E8 ± 2580378.4857196654 ns/op
mutableUniversalRefFloatIn2Longs: 1.01884019E8 ± 1765958.4109350513 ns/op
valueFloat: 2.80749904E8 ± 8948120.506571705 ns/op
valueInlineFloat: 1.76219153E7 ± 189853.64468979265 ns/op
valuePreserveBoxFloat: 2.65442077E8 ± 9930493.792832123 ns/op
baselineDouble: 3.843501915E8 ± 1.1195484698001184E7 ns/op
inlineDouble: 8.112936145E8 ± 2.9147372356726736E7 ns/op
mutableSpecificRefDouble: 8.0806442E7 ± 1347884.0904255523 ns/op
mutableUniversalRefDouble: 1.066349805E8 ± 1892386.4031551124 ns/op
valueDouble: 4.503408455E8 ± 1.0045910013593635E7 ns/op
valueInlineDouble: 1.86478884E7 ± 243807.8775418186 ns/op
valuePreserveBoxDouble: 4.2855275E8 ± 1.1082285551532617E7 ns/op

In [6]:
makePlot("Wolf3d", "ART", readArtBenchmarkData("Wolf3dBenchmark"))
makePlot("Ackermann", "ART", readArtBenchmarkData("AckermannBenchmark"))
makePlot("BoxRecreation", "ART", readArtBenchmarkData("BoxRecreationBenchmark"))